In [1]:
import fiona
import geopandas as gpd
import pandas as pd
import json
import pyrosm
import osmnx as ox
import contextily as cx
import matplotlib.pyplot as plt

In [2]:
with open('proj4_params.json', encoding="utf-8") as json_file:
    params = json.load(json_file)
CRS = "EPSG:2178"
#CRS = "EPSG:3857"
params

{'city': 'Kraków, Poland', 'id_column': 'lamp_id'}

In [3]:
#TASK 1
points = gpd.read_file("proj4_points.geojson")
#print(points.crs)


points = points.to_crs(CRS) #crs for poland representing positions in meters
#print(points.crs)
points_copy = points.copy()
print(type(points_copy))
points

<class 'geopandas.geodataframe.GeoDataFrame'>


,lamp_id,circuit,label,geometry
0,5907,4260,4260 III/9,POINT (7421200.89 5549236.163)
1,5908,4104,4104 I/7,POINT (7420648.372 5549356.622)
2,5909,4070,4070 I/8,POINT (7421133.034 5549127.243)
3,5910,4148,4148 I/14,POINT (7420454.349 5549189.001)
4,5911,4148,4148 I/4,POINT (7420665.797 5549159.122)
...,...,...,...,...
3737,9644,1025,1025 VIII/21,POINT (7423714.91 5549133.25)
3738,9645,1145,1145 I/11,POINT (7423773.12 5548803.43)
3739,9646,1145,1145 I/12,POINT (7423786.97 5548784.45)
3740,9647,1145,1145 I/13,POINT (7423800.54 5548765.9)


In [4]:
points["buffer_geom"] = points.geometry.buffer(100) #object of 100 m radius around each lamp
name_of_id_column = params["id_column"]

df_joined = gpd.sjoin(
    points[[name_of_id_column, "geometry"]],
    points.set_geometry("buffer_geom"),
    predicate='within')
#print(type(df_joined))
df_joined[:20]

,lamp_id_left,geometry,index_right,lamp_id_right,circuit,label,geometry_right
0,5907,POINT (7421200.89 5549236.163),0,5907,4260,4260 III/9,POINT (7421200.89 5549236.163)
0,5907,POINT (7421200.89 5549236.163),677,6584,4065,4065 II/23,POINT (7421115.63 5549203.24)
0,5907,POINT (7421200.89 5549236.163),678,6585,4065,4065 II/22,POINT (7421135.58 5549204.56)
0,5907,POINT (7421200.89 5549236.163),680,6587,4260,4260 III/6,POINT (7421213.33 5549320.88)
0,5907,POINT (7421200.89 5549236.163),681,6588,4260,4260 III/7,POINT (7421209.27 5549294.09)
0,5907,POINT (7421200.89 5549236.163),682,6589,4260,4260 III/8,POINT (7421204.22 5549260.34)
0,5907,POINT (7421200.89 5549236.163),683,6590,4065,4065 II/13,POINT (7421208.63 5549211.91)
0,5907,POINT (7421200.89 5549236.163),684,6591,4065,4065 II/14,POINT (7421182.03 5549209.41)
0,5907,POINT (7421200.89 5549236.163),685,6592,4065,4065 II/15,POINT (7421159.08 5549204.85)
0,5907,POINT (7421200.89 5549236.163),686,6593,4065,4065 II/12,POINT (7421231.53 5549217.93)


In [13]:
#part 1
df_counted_grouped = df_joined.groupby(name_of_id_column+"_left")[name_of_id_column+"_right"].count()
df_counts = df_counted_grouped.reset_index()
df_counts.columns = [name_of_id_column, "count"]
df_counts.to_csv("proj4_ex01_counts.csv",index=True)
df_counts

,lamp_id,count
0,5907,16
1,5908,16
2,5909,17
3,5910,20
4,5911,9
...,...,...
3737,9644,16
3738,9645,16
3739,9646,15
3740,9647,12


In [6]:


#part 2
points_with_geometry = points.merge(df_counts, on=name_of_id_column, how="left")
points_latlon = points_with_geometry.to_crs("EPSG:4326") #converting to geographic CRS - to get lat and lon
points_latlon["lon"] = points_latlon.geometry.x.round(7)
points_latlon["lat"] = points_latlon.geometry.y.round(7)
points_ready_for_csv = points_latlon.loc[:,[name_of_id_column, "lat","lon"]]
points_ready_for_csv.to_csv("proj4_ex01_coords.csv", index=False)
points_ready_for_csv

,lamp_id,lat,lon
0,5907,50.074043,19.899135
1,5908,50.075053,19.891393
2,5909,50.073055,19.898210
3,5910,50.073520,19.888718
4,5911,50.073280,19.891677
...,...,...,...
3737,9644,50.073446,19.934272
3738,9645,50.070489,19.935150
3739,9646,50.070320,19.935347
3740,9647,50.070155,19.935541


In [7]:
#TASK 2
city = params["city"]
osm_tags = {
    'highway': [
        'tertiary'
    ]
}

gdf_city = ox.features_from_place(city, tags=osm_tags)
gdf_city = gdf_city.reset_index() #to acess multiindex
gdf_city = gdf_city.loc[:,["id","name","geometry"]]
gdf_city.columns=("osm_id","name","geometry")
gdf_city.to_file('proj4_ex02_roads.geojson', index=False)
print(type(gdf_city))
gdf_city

<class 'geopandas.geodataframe.GeoDataFrame'>


,osm_id,name,geometry
0,2954556,Stefana Żeromskiego,"LINESTRING (20.03607 50.0794, 20.03613 50.0793..."
1,4757981,Księcia Józefa,"LINESTRING (19.90658 50.05036, 19.90651 50.050..."
2,4758409,Księcia Józefa,"LINESTRING (19.90589 50.04991, 19.90562 50.049..."
3,4758410,Księcia Józefa,"LINESTRING (19.90589 50.04991, 19.90523 50.049..."
4,5095912,Borkowska,"LINESTRING (19.90506 50.0071, 19.90516 50.0070..."
...,...,...,...
2488,1380087645,Balicka,"LINESTRING (19.87988 50.08221, 19.88005 50.082..."
2489,1380089315,Bronowicka,"LINESTRING (19.89237 50.0809, 19.89256 50.08086)"
2490,1380089316,Bronowicka,"LINESTRING (19.89203 50.08097, 19.89172 50.08103)"
2491,1380175078,Księcia Józefa,"LINESTRING (19.90611 50.05007, 19.90589 50.04991)"


In [8]:
#TASK 3
gdf_city = gdf_city.to_crs(CRS)
gdf_city

,osm_id,name,geometry
0,2954556,Stefana Żeromskiego,"LINESTRING (7431009.939 5549696.509, 7431014.6..."
1,4757981,Księcia Józefa,"LINESTRING (7421695.43 5546593.949, 7421690.17..."
2,4758409,Księcia Józefa,"LINESTRING (7421644.893 5546544.797, 7421625.5..."
3,4758410,Księcia Józefa,"LINESTRING (7421644.893 5546544.797, 7421597.2..."
4,5095912,Borkowska,"LINESTRING (7421516.022 5541784.864, 7421523.1..."
...,...,...,...
2488,1380087645,Balicka,"LINESTRING (7419836.201 5550165.201, 7419848.6..."
2489,1380089315,Bronowicka,"LINESTRING (7420728.156 5550005.482, 7420741.3..."
2490,1380089316,Bronowicka,"LINESTRING (7420703.373 5550013.603, 7420681.8..."
2491,1380175078,Księcia Józefa,"LINESTRING (7421660.992 5546562.694, 7421644.8..."


In [9]:
# gdf_city['buffer'] = gdf_city.geometry.buffer(50, cap_style=2)
# gdf_buffered = gpd.GeoDataFrame(gdf_city[['name']], geometry=gdf_city['buffer'], crs=gdf_city.crs)
# joined = gpd.sjoin(points_copy, gdf_buffered, predicate = "within")
# joined

#BETTER
gdf_city["buffer_street"] = gdf_city.geometry.buffer(50, cap_style=2)
gdf_city_buffered = gpd.GeoDataFrame(gdf_city[['name']], geometry=gdf_city['buffer_street'], crs=gdf_city.crs)
df_joined_street_point2 = gpd.sjoin(
    points_copy,
    gdf_city_buffered,
    predicate = "within"
)
df_joined_street_point2

# points_copy["buffer_street"] = points_copy.geometry.buffer(50,cap_style=1)
# df_joined_street_point2 = gpd.sjoin(
#     gdf_city[['name', 'geometry']],
#     points_copy.set_geometry('buffer_street'),
#     predicate = "within"
# )
# df_joined_street_point2

,lamp_id,circuit,label,geometry,index_right,name
22,5929,4099,4099 III/28,POINT (7421173.742 5549694.675),1015,Bronowicka
32,5939,4099,4099 I/8,POINT (7421270.033 5549635.812),1011,Bronowicka
33,5940,4099,4099 I/7,POINT (7421281.139 5549659.775),1015,Bronowicka
35,5942,4112,4112 II/13,POINT (7421255.566 5549748.616),1015,Bronowicka
36,5943,4112,4112 II/12,POINT (7421411.215 5549736.297),1595,Lucjana Rydla
...,...,...,...,...,...,...
3723,9630,4133,4133 II/8,POINT (7423974.04 5549711.66),1275,Prądnicka
3723,9630,4133,4133 II/8,POINT (7423974.04 5549711.66),1276,Prądnicka
3726,9633,4154,4154 I/4,POINT (7423779.83 5549256.29),39,Wrocławska
3726,9633,4154,4154 I/4,POINT (7423779.83 5549256.29),1564,Prądnicka


In [10]:
# street_counts = joined.groupby('name')[name_of_id_column].count()
# street_counts

df_street_counted_grouped = df_joined_street_point2.groupby("name")[name_of_id_column].count()
df_counts = df_street_counted_grouped.reset_index()
df_counts.columns = ["name", "point_count"]
df_counts = df_counts[df_counts["point_count"] > 0]
df_counts.to_csv("proj4_ex03_streets_points.csv",index=False)
df_counts

,name,point_count
0,Aleja 3 Maja,131
1,Aleja Kijowska,128
2,Aleja Marszałka Ferdinanda Focha,33
3,Balicka,79
4,Bartosza Głowackiego,23
5,Bronowicka,148
6,Czarnowiejska,14
7,Dolnych Młynów,12
8,Józefa Szujskiego,5
9,Karmelicka,64


In [11]:
#TASK 4
gdf_countries = gpd.read_file("proj4_countries.geojson")
gdf_countries.to_crs(epsg=3857, inplace=True)
print("No of countries:", len(gdf_countries))
gdf_countries.to_pickle("proj4_ex04_gdf.pkl")
gdf_countries

No of countries: 5


,name,geometry
0,Vietnam,"MULTIPOLYGON (((11584347.576 1163093.973, 1158..."
1,Sweden,"MULTIPOLYGON (((2123582.353 7932931.805, 21143..."
2,Poland,"POLYGON ((2627727.019 6713424.195, 2633695.222..."
3,Italy,"MULTIPOLYGON (((781584.581 5768463.58, 785443...."
4,Chile,"MULTIPOLYGON (((-12164991.78 -3141027.565, -12..."


In [12]:
for i, row in gdf_countries.iterrows():
    country_name = row["name"]
    country_geometry = gpd.GeoDataFrame([row], crs=gdf_countries.crs)
    
    #facecolor none removes fill
    #edgecolor=blue outlines shapes
    ax = country_geometry.plot(edgecolor='blue', facecolor='none', linewidth=2)

    cx.add_basemap(ax, crs=gdf_countries.crs.to_string())

    fig = ax.get_figure()
    filename = f"proj4_ex04_{country_name.lower().replace(' ', '_')}.png"
    fig.savefig(filename)
    plt.close(fig)